# Quickstart with BentoML and Scikit-learn

Link to source code: https://github.com/bentoml/gallery/tree/main/quickstart

Install required dependencies:

In [ ]:
!pip install -r requirements.txt

In [ ]:
from sklearn import svm, datasets

### Train a classifier model using the iris dataset:

In [ ]:
# Load training data
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Model Training
clf = svm.SVC()
clf.fit(X, y)

Save the `clf` model with BentoML:

In [ ]:
import bentoml

bentoml.sklearn.save("iris_clf", clf)

Models saved can be accessed via `bentoml models` CLI command:

In [ ]:
!bentoml models list

Test loading the model as a BentoML Runner instance:

In [ ]:
test_runner = bentoml.sklearn.load_runner("iris_clf:latest")
test_runner.run([5.9, 3. , 5.1, 1.8])  # => array(2)

### Create a BentoML Service for serving the model

Note: using `%%writefile` here because `bentoml.Service` instance must be created in a separate `.py` file

In [ ]:
%%writefile service.py
import numpy as np
import bentoml
from bentoml.io import NumpyNdarray


iris_clf_runner = bentoml.sklearn.load_runner("iris_clf:latest")

svc = bentoml.Service("iris_classifier", runners=[iris_clf_runner])


@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_series: np.ndarray) -> np.ndarray:
    return iris_clf_runner.run(input_series)

Start a dev model server to test out the service defined above:

In [ ]:
!bentoml serve service.py:svc

Open a new browser tab and test out the API endpoint from the web UI.

### Build a Bento for distribution and deployment

In [ ]:
bentoml.build(
    "service.py:svc",
    include=["*.py"],
    description="file:./README.md",
    python=dict(
        packages=["scikit-learn"]
    )
)

Starting a dev server with the Bento build:

In [ ]:
!bentoml serve iris_classifier:latest

## Hooray, your model is ready for production now 🎉

Bento is a standarlized format for storing models alongside with all its API definitions, configurations, and environment settings. BentoML can start an API server serving a Bento over REST API, run a Bento as batch processing job on distributed dataset, and containerize all dependencies and models into a docker container image for easy production deployment.

### Optional: create a docker image for the model server

This will require docker cli to be installed and docker deamon running:

In [ ]:
bentoml.bentos.containerize("iris_classifier:latest")

## What's next?

* Learn more at http://docs.bentoml.org 📖
* Join [BentoML Slack community](https://join.slack.com/t/bentoml/shared_invite/enQtNjcyMTY3MjE4NTgzLTU3ZDc1MWM5MzQxMWQxMzJiNTc1MTJmMzYzMTYwMjQ0OGEwNDFmZDkzYWQxNzgxYWNhNjAxZjk4MzI4OGY1Yjg) 👈
* Follow us on [Twitter](https://twitter.com/bentomlai) 🐦
* Contribute to [BentoML on Github](https://github.com/bentoml/BentoML) 💻